# Import Libraries

In [1]:
from tqdm import tqdm
import pandas as pd
import nltk
from nltk import word_tokenize
import string

from utils import load_sessions, read_session
from main import generate_buffer
from events import generate_event_seq
from summary import stats

# Compute summary statistics

In [2]:
sessions = load_sessions()
# sessions = load_sessions()[:10]

file_name = []
text = []
sentence_metrics_list = []
api_metrics_list = []

err = []

for sess in tqdm(sessions):
    events = read_session(sess, verbose=0)
    try:
        text_buffer = generate_buffer(events)
    except:
        err.append(str(sess.split('/')[-1]) + " is throwing an error!")
        continue
    file_name.append(sess.split('/')[-1])
    text.append(text_buffer[-1])
    event_seq_dict = generate_event_seq(buffer=text_buffer,
                                        events=events)
    sentence_metrics, api_metrics = stats(event_seq_dict)
    sentence_metrics_list.append(sentence_metrics)
    api_metrics_list.append(api_metrics)
    
for e in err:
    print(e)
    
df = pd.DataFrame()

df["file_name"] = file_name
df["text"] = text

for col in sentence_metrics_list[0]:
    df[str(col)] = [x[col] for x in sentence_metrics_list]
    
for col in api_metrics_list[0]:
    df[str(col)] = [x[col] for x in api_metrics_list]

Successfully downloaded 1447 writing sessions in CoAuthor!


 51%|███████████████▏              | 734/1447 [10:32<11:15,  1.06it/s]/home/sri/Documents/UTS Research/coauthor-visualizer/summary.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ("gpt3-call" not in seq) and ("prompt" not in seq) and ("user" in seq):
/home/sri/Documents/UTS Research/coauthor-visualizer/summary.py:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ("gpt3-call" in seq) and ("user" not in seq):
 54%|████████████████▏             | 782/1447 [11:11<11:03,  1.00it/s]/home/sri/Documents/UTS Research/coauthor-visualizer/summary.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ("gpt3-call" not in seq) and ("prompt" not in seq) and ("user" in seq):
/home/sri/Documents/UTS Research/coauthor-visualizer/summary.py:19: Futur

312e3263a9f24f3184364949a42a6dfc.jsonl is throwing an error!


# Ratios

In [3]:
def get_ratio(num1, num2):
    return float(num1 / num2)

In [4]:
# GPT-3 : Total Sentences

df["GPT-3 : Total Sentences"] = list(map(get_ratio, 
    df["Number of sentences completely authored by GPT-3"], 
    df["Total number of sentences"]
))

df["GPT-3 : Total Sentences"]

0       0.00000
1       0.00000
2       0.00000
3       0.00000
4       0.02381
         ...   
1441    0.00000
1442    0.00000
1443    0.00000
1444    0.00000
1445    0.00000
Name: GPT-3 : Total Sentences, Length: 1446, dtype: float64

In [5]:
# User : Total Sentences

df["User : Total Sentences"] = list(map(get_ratio, 
    df["Number of sentences completely authored by the user"], 
    df["Total number of sentences"]
))

df["User : Total Sentences"]

0       0.451613
1       0.055556
2       0.451613
3       0.730769
4       0.190476
          ...   
1441    0.375000
1442    0.555556
1443    0.758621
1444    0.526316
1445    0.263158
Name: User : Total Sentences, Length: 1446, dtype: float64

In [6]:
# Type Token Ratio

def get_ttr(text):
    sentence_tokens = word_tokenize(text)
    punctuations = list(string.punctuation)
    sentence_tokens_clean = [word for word in sentence_tokens if word not in punctuations]
    ttr = len(set(sentence_tokens_clean)) / len(sentence_tokens_clean)
    return ttr


df["Type Token Ratio"] = df["text"].apply(get_ttr)
df["Type Token Ratio"]

0       0.561713
1       0.555556
2       0.513580
3       0.513064
4       0.348011
          ...   
1441    0.476965
1442    0.503185
1443    0.470260
1444    0.497024
1445    0.466238
Name: Type Token Ratio, Length: 1446, dtype: float64

# Mean

In [7]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Mean of", col, ":", np.mean(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Mean of", col, ":", np.mean(df[col]))
    
print("\nRatios")
print("Mean of GPT-3 / Total Sentences : ", np.mean(df["GPT-3 : Total Sentences"]))
print("Mean of User / Total Sentences : ", np.mean(df["User : Total Sentences"]))
print("Mean of Type Token Ratio : ", np.mean(df["Type Token Ratio"]))

Sentence Metrics
Mean of Total number of sentences : 28.96265560165975
Mean of Number of sentences of initial prompt : 4.421161825726141
Mean of Number of sentences completely authored by the user : 16.24273858921162
Mean of Number of sentences completely authored by GPT-3 : 0.6853388658367912
Mean of Number of sentences authored by GPT-3 and user : 7.6134163208852

API Metrics
Mean of Total number of GPT-3 calls made : 12.531120331950207
Mean of Number of times GPT-3 suggestion is used : 8.857538035961273
Mean of Number of times user rejected GPT-3 suggestion : 3.673582295988935
Mean of Number of times GPT-3 suggestion is modified : 1.586445366528354
Mean of Number of times GPT-3 suggestion is used as is : 7.271092669432918

Ratios
Mean of GPT-3 / Total Sentences :  0.021900159513948938
Mean of User / Total Sentences :  0.5412613590031222
Mean of Type Token Ratio :  0.48183805678712555


# Median

In [8]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Median of", col, ":", np.median(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Median of", col, ":", np.median(df[col]))

print("\nRatios")
print("Median of GPT-3 / Total Sentences : ", np.median(df["GPT-3 : Total Sentences"]))
print("Median of User / Total Sentences : ", np.median(df["User : Total Sentences"]))
print("Median of Type Token Ratio : ", np.median(df["Type Token Ratio"]))

Sentence Metrics
Median of Total number of sentences : 27.0
Median of Number of sentences of initial prompt : 4.0
Median of Number of sentences completely authored by the user : 15.0
Median of Number of sentences completely authored by GPT-3 : 0.0
Median of Number of sentences authored by GPT-3 and user : 6.0

API Metrics
Median of Total number of GPT-3 calls made : 10.0
Median of Number of times GPT-3 suggestion is used : 7.0
Median of Number of times user rejected GPT-3 suggestion : 3.0
Median of Number of times GPT-3 suggestion is modified : 1.0
Median of Number of times GPT-3 suggestion is used as is : 5.0

Ratios
Median of GPT-3 / Total Sentences :  0.0
Median of User / Total Sentences :  0.5641025641025641
Median of Type Token Ratio :  0.4821182605273514


# Standard Deviation

In [9]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Standard Deviation of", col, ":", np.std(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Standard Deviation of", col, ":", np.std(df[col]))
    
print("\nRatios")
print("Standard Deviation of GPT-3 / Total Sentences : ", np.std(df["GPT-3 : Total Sentences"]))
print("Standard Deviation of User / Total Sentences : ", np.std(df["User : Total Sentences"]))
print("Standard Deviation of Type Token Ratio : ", np.std(df["Type Token Ratio"]))

Sentence Metrics
Standard Deviation of Total number of sentences : 10.388909909258523
Standard Deviation of Number of sentences of initial prompt : 2.3909859029112486
Standard Deviation of Number of sentences completely authored by the user : 9.535179050568926
Standard Deviation of Number of sentences completely authored by GPT-3 : 1.8864423445191325
Standard Deviation of Number of sentences authored by GPT-3 and user : 5.953072577616293

API Metrics
Standard Deviation of Total number of GPT-3 calls made : 9.204158194377401
Standard Deviation of Number of times GPT-3 suggestion is used : 7.424057788661343
Standard Deviation of Number of times user rejected GPT-3 suggestion : 3.530339833311101
Standard Deviation of Number of times GPT-3 suggestion is modified : 1.796857239727531
Standard Deviation of Number of times GPT-3 suggestion is used as is : 7.233591709071117

Ratios
Standard Deviation of GPT-3 / Total Sentences :  0.054147218196700535
Standard Deviation of User / Total Sentences

# Minimum

In [10]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Minimum of", col, ":", np.min(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Minimum of", col, ":", np.min(df[col]))
    
print("\nRatios")
print("Minimum of GPT-3 / Total Sentences : ", np.min(df["GPT-3 : Total Sentences"]))
print("Minimum of User / Total Sentences : ", np.min(df["User : Total Sentences"]))
print("Minimum of Type Token Ratio : ", np.min(df["Type Token Ratio"]))

Sentence Metrics
Minimum of Total number of sentences : 11
Minimum of Number of sentences of initial prompt : 0
Minimum of Number of sentences completely authored by the user : 0
Minimum of Number of sentences completely authored by GPT-3 : 0
Minimum of Number of sentences authored by GPT-3 and user : 0

API Metrics
Minimum of Total number of GPT-3 calls made : 0
Minimum of Number of times GPT-3 suggestion is used : 0
Minimum of Number of times user rejected GPT-3 suggestion : 0
Minimum of Number of times GPT-3 suggestion is modified : 0
Minimum of Number of times GPT-3 suggestion is used as is : 0

Ratios
Minimum of GPT-3 / Total Sentences :  0.0
Minimum of User / Total Sentences :  0.0
Minimum of Type Token Ratio :  0.28794178794178793


# Maximum

In [11]:
import numpy as np

print("Sentence Metrics")
for col in sentence_metrics_list[0]:
    print("Maximum of", col, ":", np.max(df[col]))
    
print("\nAPI Metrics")
for col in api_metrics_list[0]:
    print("Maximum of", col, ":", np.max(df[col]))
    
print("\nRatios")
print("Maximum of GPT-3 / Total Sentences : ", np.max(df["GPT-3 : Total Sentences"]))
print("Maximum of User / Total Sentences : ", np.max(df["User : Total Sentences"]))
print("Maximum of Type Token Ratio : ", np.max(df["Type Token Ratio"]))

Sentence Metrics
Maximum of Total number of sentences : 78
Maximum of Number of sentences of initial prompt : 9
Maximum of Number of sentences completely authored by the user : 64
Maximum of Number of sentences completely authored by GPT-3 : 22
Maximum of Number of sentences authored by GPT-3 and user : 42

API Metrics
Maximum of Total number of GPT-3 calls made : 65
Maximum of Number of times GPT-3 suggestion is used : 47
Maximum of Number of times user rejected GPT-3 suggestion : 24
Maximum of Number of times GPT-3 suggestion is modified : 10
Maximum of Number of times GPT-3 suggestion is used as is : 47

Ratios
Maximum of GPT-3 / Total Sentences :  0.6111111111111112
Maximum of User / Total Sentences :  0.9629629629629629
Maximum of Type Token Ratio :  0.6879432624113475


# Correlation

In [12]:
df.corr()

/tmp/ipykernel_51147/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Total number of sentences,Number of sentences of initial prompt,Number of sentences completely authored by the user,Number of sentences completely authored by GPT-3,Number of sentences authored by GPT-3 and user,Total number of GPT-3 calls made,Number of times GPT-3 suggestion is used,Number of times user rejected GPT-3 suggestion,Number of times GPT-3 suggestion is modified,Number of times GPT-3 suggestion is used as is,GPT-3 : Total Sentences,User : Total Sentences,Type Token Ratio
Total number of sentences,1.000000,-0.129523,0.773865,0.218429,0.488421,0.365100,0.454037,-0.002934,0.051890,0.453102,0.090754,0.265854,-0.527044
Number of sentences of initial prompt,-0.129523,1.000000,-0.300117,-0.062613,-0.127129,-0.067734,-0.096746,0.026855,-0.056362,-0.085293,-0.060789,-0.377381,0.034719
Number of sentences completely authored by the user,0.773865,-0.300117,1.000000,-0.083143,-0.104341,-0.218031,-0.167396,-0.216420,-0.100297,-0.146890,-0.181537,0.779995,-0.291106
Number of sentences completely authored by GPT-3,0.218429,-0.062613,-0.083143,1.000000,0.222622,0.422258,0.451338,0.151763,0.199498,0.413665,0.945054,-0.281658,-0.171809
Number of sentences authored by GPT-3 and user,0.488421,-0.127129,-0.104341,0.222622,1.000000,0.879771,0.956312,0.282646,0.210622,0.929173,0.174091,-0.544562,-0.412992
Total number of GPT-3 calls made,0.365100,-0.067734,-0.218031,0.422258,0.879771,1.000000,0.931990,0.647249,0.192459,0.908722,0.395012,-0.610905,-0.358701
Number of times GPT-3 suggestion is used,0.454037,-0.096746,-0.167396,0.451338,0.956312,0.931990,1.000000,0.326916,0.225656,0.970277,0.407231,-0.609296,-0.404455
Number of times user rejected GPT-3 suggestion,-0.002934,0.026855,-0.216420,0.151763,0.282646,0.647249,0.326916,1.000000,0.027233,0.328759,0.173480,-0.311419,-0.084651
Number of times GPT-3 suggestion is modified,0.051890,-0.056362,-0.100297,0.199498,0.210622,0.192459,0.225656,0.027233,1.000000,-0.016807,0.184228,-0.200982,-0.053334
Number of times GPT-3 suggestion is used as is,0.453102,-0.085293,-0.146890,0.413665,0.929173,0.908722,0.970277,0.328759,-0.016807,1.000000,0.372191,-0.575415,-0.401856


# Export to CSV

In [13]:
df.to_csv("writing_session_stats.csv")